In [15]:
import mysql.connector

# Cria conexão com o banco de dados
cnx = mysql.connector.connect(
    host='localhost',
    user='root',
    password='alecrim',
    database='alecrim_normalized'
)

# Cria tabela cardapio
cursor = cnx.cursor()

query = '''
    CREATE TABLE cardapio (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        categoria VARCHAR(50) NOT NULL
    )
'''
cursor.execute(query)

# Cria tabela caixa
query = '''
    CREATE TABLE caixa (
        id INT AUTO_INCREMENT PRIMARY KEY,
        operacao VARCHAR(50) NOT NULL
    )
'''
cursor.execute(query)

# Cria tabela vendas
query = '''
    CREATE TABLE vendas (
        id INT AUTO_INCREMENT PRIMARY KEY,
        id_caixa INT,
        FOREIGN KEY (id_caixa) REFERENCES caixa(id)
    )
'''
cursor.execute(query)

# Cria tabela estoque
query = '''
    CREATE TABLE estoque (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        quantidade INT NOT NULL,
        unidade VARCHAR(10) NOT NULL
    )
'''
cursor.execute(query)

# cria tabela estoque_lote_processado
query = '''
    CREATE TABLE estoque_lote_processado (
        id INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
        nome VARCHAR(50),
        quantidade INT
    )
'''
cursor.execute(query)

# Cria tabela compras
query = '''
    CREATE TABLE compras (
        id INT AUTO_INCREMENT PRIMARY KEY,
        data DATE,
        id_caixa INT NOT NULL,
        prazo INT,
        forma_pagamento VARCHAR(50) NOT NULL,
        FOREIGN KEY (id_caixa) REFERENCES caixa(id)
    )
'''
cursor.execute(query)

# Cria tabela compras_itens
query = '''
    CREATE TABLE compras_itens (
        id_compra INT NOT NULL,
        id_estoque INT NOT NULL,
        quantidade INT,
        custo_unitario DECIMAL(10, 2),
        FOREIGN KEY (id_compra) REFERENCES compras(id),
        FOREIGN KEY (id_estoque) REFERENCES estoque(id)
    )
'''
cursor.execute(query)

# Cria tabela ficha_tecnica
query = '''
    CREATE TABLE ficha_tecnica (
        id_estoque INT,
        id_cardapio INT,
        id_lote INT,
        obs_item VARCHAR(50),
        descricao VARCHAR(200),
        quantidade INT NOT NULL,
        tempo_preparo TIME NOT NULL,
        FOREIGN KEY (id_cardapio) REFERENCES cardapio(id),
        FOREIGN KEY (id_estoque) REFERENCES estoque(id),
        FOREIGN KEY (id_lote) REFERENCES estoque_lote_processado(id)
    )
'''
cursor.execute(query)


# Cria tabela vendas produto
query = '''
    CREATE TABLE vendas_produto (
        id_venda INT,
        id_cardapio INT,
        quantidade INT NOT NULL,
        forma_pagamento VARCHAR(50) NOT NULL,
        hora TIME,
        data DATE,
        FOREIGN KEY (id_cardapio) REFERENCES cardapio(id),
        FOREIGN KEY (id_venda) REFERENCES vendas(id)
    )
'''
cursor.execute(query)


# cria tabela ficha_tecnica_lote_processado
query = '''
    CREATE TABLE ficha_tecnica_lote_processado (
        id_estoque INT NOT NULL,
        id_lote INT NOT NULL,
        rendimento VARCHAR(50),
        obs_item VARCHAR(50),
        quantidade INT,
        tempo_preparo TIME NOT NULL,
        FOREIGN KEY (id_lote) REFERENCES estoque_lote_processado(id),
        FOREIGN KEY (id_estoque) REFERENCES estoque(id)
    )
'''
cursor.execute(query)

# fecha conexão
cnx.close()



In [ ]:
from flask import Flask, request, jsonify
import mysql.connector

try:
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="123456",
        database="alecrim"
    )
    print('Conexão com banco de dados bem sucedida.')
except:
    print('Conexão com banco de dados não sucedida.')

app = Flask(__name__)


cursor = mydb.cursor()

# rotas de cardapio

# Todos itens do cardapio


@app.route('/cardapio', methods=['GET'])
def tabela_cardapio():
    cursor.execute("SELECT * FROM cardapio")
    resultado = cursor.fetchall()
    return jsonify(resultado)

# Item do cardapio por id


@app.route('/cardapio/<int:id>', methods=['GET'])
def tabela_cardapio_item(id):
    cursor.execute("SELECT * FROM cardapio WHERE id = %s", (id,))
    item = cursor.fetchone()
    if item is None:
        return jsonify({'mensagem': 'Registro não encontrado'}), 404
    return jsonify(item)

# Adicionar item no cardapio


@app.route('/cardapio', methods=['POST'])
def tabela_cardapio_adicionar_item():
    data = request.get_json()
    cursor.execute("INSERT INTO cardapio (id_produto, nome_produto, descricao_prato, preco_venda, categoria) VALUES (%s, %s, %s, %s, %s)",
                   (data['id_produto'], data['nome_produto'], data['descricao_prato'], data['preco_venda'], data['categoria']))
    mydb.commit()
    return jsonify({'mensagem': 'Registro adicionado com sucesso ao cardápio'})


# Atualizar um item do cardapio
@app.route('/cardapio/<int:id_produto>', methods=['PUT'])
def tabala_cardapio_atualizar_item(id_produto):
    data = request.get_json()
    cursor.execute("UPDATE cardapio SET id_produto=%s, nome_produto=%s, descricao_prato=%s, preco_venda=%s, categoria=%s WHERE id=%s",
                   (data['id_produto'], data['nome_produto'], data['descricao_prato'], data['preco_venda'], data['categoria'], id_produto))
    mydb.commit()
    return jsonify({'mensagem': 'Registro atualizado com sucesso'})

# Excluir um item do cardapio


@app.route('/cardapio/<int:id_produto>', methods=['DELETE'])
def tabala_cardapio_excluir_item(id_produto):
    cursor.execute("DELETE FROM cardapio WHERE id_produto = %s", (id_produto,))
    mydb.commit()
    return jsonify({'mensagem': 'Registro excluído com sucesso'})


# rotas de clientes

@app.route('/clientes')
def tabela_clientes():
    cursor.execute("SELECT * FROM clientes")
    resultado = cursor.fetchall()
    return jsonify(resultado)


# Iniciando aplicação Flask
if __name__ == '__main__':
    app.run(debug=True)